In [2]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [3]:
#Importación de selenium 

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
from datetime import datetime
from collections import defaultdict
from bs4 import BeautifulSoup

#Importación de beautifulsoup
from bs4 import BeautifulSoup
import requests

#Otros
from IPython.display import clear_output
import pandas as pd
import time
import pickle
import logging
import threading
import random
import numpy as np
#Paquetes personalizados
from policy_evaluation_class import Politica


## Parámetros globales

In [4]:
threshold_average_win_para_apostar = 0.66
threshold_odd_para_apostar = 2

gamma = 0.04
gamma_lose = 0.08
umbral = 0.7

In [5]:
#Clase para activar listener
class history_has_changed(object):
 
  def __init__(self, texto_anterior):
    self.texto_anterior = texto_anterior

  def __call__(self, driver):
    element = driver.find_element_by_class_name(crash_history)   # Finding the referenced element
    if self.texto_anterior != element.text:
        global elemento_inicial_texto
        elemento_inicial_texto = element.text
        return element
    else:
        return False



In [6]:
def write_pickle_file(path, object_to_save, file_name,print_result=False):
    with open(path + file_name + '.data','wb') as filehandle:
        pickle.dump(object_to_save,filehandle)
    
    if print_result:
        print(f"The file {path}{file_name}.data was created successfully")

def read_pickle_file(path, file_name):
    with open(path + file_name + '.data', 'rb') as filehandle:
        loaded_file = pickle.load(filehandle)
    return loaded_file

In [7]:
def exportar_odds_history(new_odd):
    with open('odds_history.txt','a+') as myfile:
        myfile.write(new_odd)
        myfile.write(" ")
        myfile.write(str(datetime.now()))
        myfile.write("\n")
            
def retorna_franja_horaria():
    now = datetime.now()
    hora = now.hour
    franja = 1 if hora < 6 else 2 if hora < 12 else 3 if hora < 18 else 4
    return franja

In [8]:
def retornar_estados(odds_available,threshold):
    odds_binary = [str(int(float(odd)>threshold)) for odd in odds_available]
    
    franja = retorna_franja_horaria()
    
    estado_anterior = ''.join(odds_binary[1:8])+"-"+str(franja)
    estado_actual = ''.join(odds_binary[0:7])+"-"+str(franja)
    
    return estado_anterior, estado_actual

In [9]:
def find_jss_elements (candidates):
    candidate_bet_tags = [tag for tag in candidates if "Bet Amount½2×MaxAuto" in tag.text]

    #in tag.text and next(tag.descendants)["class"][0][:3]!="jss"
    
    candidate_crash_history_tags = [tag for tag in candidates if ("x" in tag.text) 
                                    and ("a" not in tag.text) and (len(tag.text.split('x'))>5)]
    
    bet_container_class_name = candidate_bet_tags[-1]["class"][0]
    crash_history_class_name = candidate_crash_history_tags[-1]["class"][0]
    
    return bet_container_class_name,crash_history_class_name


In [11]:
driver.quit()

In [10]:
url = "https://roobet.com/crash"
chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument("--incognito")
#driver = webdriver.Chrome("chromedriver_win32/chromedriver",chrome_options=chrome_options)
driver = webdriver.Chrome("chromedriver_win32/chromedriver")
driver.get(url)
time.sleep(5)
#Find jss elements
soup = BeautifulSoup(driver.page_source,'html.parser')
candidate_tags = soup.findAll('div',{'class': lambda x: str(x)[:3]=="jss"})
bet_button_container,crash_history = find_jss_elements(candidates= candidate_tags)

In [10]:
#Validar elementos iniciales
elemento_inicial = driver.find_element_by_class_name(crash_history)
elemento_inicial_texto = elemento_inicial.text

bet_box = driver.find_element_by_class_name(bet_button_container)
print(bet_box.text)
elemento_inicial_texto

Bet Amount
½
2×
Max
Auto Cashout
Place Bet


'6.71x\n2.30x\n2.30x\n1.71x\n55.15x\n172.01x\n1.76x\n1.00x\n1.93x\n2.41x\n1.46x\n1.23x\n4.36x\n1.38x\n1.38x\n1.07x\n1.78x\n1.17x\n1.00x'

## Cargue de la política inicial

In [11]:
politica = read_pickle_file(path="", file_name="politica")
total_multiplicador=0

In [12]:
#new_odd=elemento_inicial_texto.split('x\n')[0:8][0]

#new_odd
#float(new_odd) if float(new_odd)>20 else -float(new_odd)


In [15]:
element.text

'10.72x\n4.59x\n9.50x\n6.71x\n2.30x\n2.30x\n1.71x\n55.15x\n172.01x\n1.76x\n1.00x\n1.93x\n2.41x\n1.46x\n1.23x\n4.36x\n1.38x'

In [16]:
odds_available

['10.72', '4.59', '9.50', '6.71', '2.30', '2.30', '1.71', '55.15']

In [19]:
politica = read_pickle_file(path="", file_name="politica")
nombre_archivo = datetime.now().strftime('%c').replace(":",'')
lista_date = []
lista_odds = [] 
lista_policy = []
lista_time = [] 
total_multiplicador=0
#Definir monto apuesta y cashout
place_bet_container = driver.find_element_by_class_name(bet_button_container)
contenedor_monto_apuesta = place_bet_container.find_elements_by_class_name("MuiGrid-item")[0]
contenedor_cashout = place_bet_container.find_elements_by_class_name("MuiGrid-item")[1]

#textbox_cashout = contenedor_cashout.find_element_by_class_name("MuiInputBase-input")
#textbox_cashout.send_keys(Keys.CONTROL + "a")
#textbox_cashout.send_keys("2,0")
#
textbox_monto_apuesta = contenedor_monto_apuesta.find_element_by_class_name("MuiInputBase-input")
textbox_monto_apuesta.send_keys(Keys.CONTROL + "a")
textbox_monto_apuesta.send_keys("0.0001")

while True:
    try:
        wait = WebDriverWait(driver, 700)
        element = wait.until(history_has_changed( elemento_inicial_texto))
        #clear_output(wait=True)
        odds_available = element.text.split('x\n')[0:8]
        #Guardar odd en archivo externo
        new_odd = odds_available[0]
        #if new_odd == odds_available[0] and new_odd != 1:
         #   #time.sleep(10)
         #   print(f"Hay una falla en la página, los odds son:{odds_available}")
         #   time.sleep(20)
         #   continue
       
        
        total_multiplicador=total_multiplicador+(10 if float(new_odd)>10 else -1)
        exportar_odds_history(new_odd)
        #Calcular win or lose
        win_or_lose = [int(float(new_odd)>threshold+1) for threshold in politica.profit]
        print(f"win or lose es:{win_or_lose}  total_multiplicador es: {total_multiplicador} el odd fue: {new_odd}")
        estado_anterior, estado_actual = retornar_estados(odds_available,threshold_odd_para_apostar)
        #Actualizar política de estado anterior
        politica.actualizar_estado(estado_anterior,gamma,gamma_lose,win_or_lose)
       
        lista_odds.append(new_odd) 
        lista_time.append(datetime.now().strftime('%H:%M:%S'))
        results={"odds":lista_odds,"time":lista_time}
        df = pd.DataFrame.from_dict(results)
        df.to_csv("resultados/Results_" +nombre_archivo+".txt", index=False)

        

        #Definir política para el periodo actual
        lista_profits_esperados = politica.df_politica.loc[estado_actual,"profit_esperado"]
        indice_max_profit= max(zip(lista_profits_esperados, range(len(lista_profits_esperados))))[1]
        
        average_win_umbral=[]
        for index,x in enumerate(politica.df_politica.loc[estado_actual,"average_win"]):
            if x>umbral:
                average_win_umbral.append(index)
        if len(average_win_umbral)>0:  
            #index_random_umbral=random.choice(average_win_umbral)
            index_random_umbral=max(average_win_umbral)
            threshold_auto_cashout = politica.profit[index_random_umbral]+1
            print(threshold_auto_cashout)
            
            place_bet_container = driver.find_element_by_class_name(bet_button_container)
            contenedor_cashout = place_bet_container.find_elements_by_class_name("MuiGrid-item")[1]

            textbox_cashout = contenedor_cashout.find_element_by_class_name("MuiInputBase-input")
            textbox_cashout.send_keys(Keys.CONTROL + "a")
            textbox_cashout.send_keys(str(threshold_auto_cashout))
           
        
        #Greedy approach. To bet or not to bet?
        
        print("\t")
        average_win_string=politica.df_politica.loc[estado_actual,"average_win"]
        print(f"Los average win del estado {estado_actual} son:{list(np.around(average_win_string,2))}")
        
        
        #average_win_argmax = politica.df_politica.loc[estado_actual,"average_win"][index_random_umbral]
        #aleatorio = random()
        if len(average_win_umbral)>0:
            apuesto = 1   
            politica.df_politica.loc[estado_actual,"apuesto"] = apuesto
            print("\t")
            average_win_selected=(politica.df_politica.loc[estado_actual,"average_win"][index_random_umbral])
            print(f"Si apuesto con el estado {estado_actual}. Su average win es {round(average_win_selected,2)} y su auto cashout es {threshold_auto_cashout}"  )

        else:
            apuesto = 0 
            politica.df_politica.loc[estado_actual,"apuesto"] = apuesto
            print("\t")
            print(f"No apuesto con el estado {estado_actual}")
        
        if apuesto:
            time.sleep(1)
            place_bet_container = driver.find_element_by_class_name(bet_button_container)
            place_bet_button = place_bet_container.find_elements_by_class_name("MuiButton-label")[-1]
            place_bet_button.click()
            
        

    except KeyboardInterrupt:   
       print("Keyboard interrupted")
       break

    except:   
        print("**************Dealing with exception*****************")   
  
        driver.get(url) 
        time.sleep(30)
        
        #Volver a buscar elementos jss
        soup = BeautifulSoup(driver.page_source,'html.parser')
        candidate_tags = soup.findAll('div',{'class': lambda x: str(x)[:3]=="jss"})
        bet_button_container,crash_history = find_jss_elements(candidates= candidate_tags)
        
        elemento_inicial = driver.find_element_by_class_name(crash_history)
        elemento_inicial_texto = elemento_inicial.text
        
        place_bet_container = driver.find_element_by_class_name(bet_button_container)
        contenedor_monto_apuesta = place_bet_container.find_elements_by_class_name("MuiGrid-item")[0]
        contenedor_cashout = place_bet_container.find_elements_by_class_name("MuiGrid-item")[1]
        
    finally:
        write_pickle_file(path="", object_to_save=politica, file_name="politica")

win or lose es:[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  total_multiplicador es: -1 el odd fue: 1.00
el estado 0001100-3 tenía un average win de: 0.12 y pasó a ser: 0.08
1.6
	
Los average win del estado 0000110-3 son:[1.0, 1.0, 1.0, 0.78, 0.6, 0.74, 0.19, 0.16, 0.04, 0.04]
	
Si apuesto con el estado 0000110-3. Su average win es 0.74 y su auto cashout es 1.6
win or lose es:[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]  total_multiplicador es: 9 el odd fue: 36.20
el estado 1111100-3 tenía un average win de: 1.00 y pasó a ser: 1.00
	
Los average win del estado 1111110-3 son:[0.65, 0.53, 0.45, 0.33, 0.33, 0.62, 0.12, 0.12, 0.12, 0.08]
	
No apuesto con el estado 1111110-3
win or lose es:[1, 1, 1, 1, 1, 0, 0, 0, 0, 0]  total_multiplicador es: 8 el odd fue: 1.59
el estado 1111110-3 tenía un average win de: 0.65 y pasó a ser: 0.69
1.1
	
Los average win del estado 0111111-3 son:[0.89, 0.37, 0.13, 0.13, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
	
Si apuesto con el estado 0111111-3. Su average win es 0.89 y su auto cashout es 1.1
wi

	
Los average win del estado 1100110-3 son:[0.93, 0.81, 0.57, 0.57, 0.57, 0.85, 0.45, 0.3, 0.16, 0.16]
	
Si apuesto con el estado 1100110-3. Su average win es 0.85 y su auto cashout es 1.6
win or lose es:[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]  total_multiplicador es: -17 el odd fue: 2.15
el estado 1100110-3 tenía un average win de: 0.93 y pasó a ser: 0.97
1.6
	
Los average win del estado 1110011-3 son:[0.93, 0.81, 0.75, 0.63, 0.63, 0.92, 0.2, 0.08, 0.08, 0.08]
	
Si apuesto con el estado 1110011-3. Su average win es 0.92 y su auto cashout es 1.6
win or lose es:[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]  total_multiplicador es: -18 el odd fue: 3.51
el estado 1110011-3 tenía un average win de: 0.93 y pasó a ser: 0.97
1.1
	
Los average win del estado 1111001-3 son:[0.79, 0.59, 0.34, 0.14, 0.04, 0.21, 0.04, 0.0, 0.0, 0.0]
	
Si apuesto con el estado 1111001-3. Su average win es 0.79 y su auto cashout es 1.1
win or lose es:[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]  total_multiplicador es: -19 el odd fue: 2.01
el estado 111

	
Los average win del estado 0110101-3 son:[0.77, 0.77, 0.77, 0.71, 0.65, 0.84, 0.45, 0.31, 0.31, 0.19]
	
Si apuesto con el estado 0110101-3. Su average win es 0.84 y su auto cashout es 1.6
win or lose es:[1, 1, 1, 1, 1, 1, 1, 0, 0, 0]  total_multiplicador es: -30 el odd fue: 1.79
el estado 0110101-3 tenía un average win de: 0.77 y pasó a ser: 0.81
	
Los average win del estado 0011010-3 son:[0.44, 0.44, 0.36, 0.36, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
	
No apuesto con el estado 0011010-3
win or lose es:[1, 1, 1, 1, 1, 0, 0, 0, 0, 0]  total_multiplicador es: -31 el odd fue: 1.54
el estado 0011010-3 tenía un average win de: 0.44 y pasó a ser: 0.48
	
Los average win del estado 0001101-3 son:[0.44, 0.28, 0.2, 0.2, 0.12, 0.27, 0.04, 0.04, 0.04, 0.04]
	
No apuesto con el estado 0001101-3
win or lose es:[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]  total_multiplicador es: -32 el odd fue: 3.78
el estado 0001101-3 tenía un average win de: 0.44 y pasó a ser: 0.48
	
Los average win del estado 1000110-3 son:[0.58, 0.5

win or lose es:[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  total_multiplicador es: -54 el odd fue: 1.08
el estado 0001010-3 tenía un average win de: 0.94 y pasó a ser: 0.86
1.6
	
Los average win del estado 0000101-3 son:[0.93, 0.75, 0.51, 0.51, 0.39, 0.78, 0.39, 0.06, 0.0, 0.0]
	
Si apuesto con el estado 0000101-3. Su average win es 0.78 y su auto cashout es 1.6
win or lose es:[1, 1, 1, 1, 1, 1, 0, 0, 0, 0]  total_multiplicador es: -55 el odd fue: 1.66
el estado 0000101-3 tenía un average win de: 0.93 y pasó a ser: 0.97
1.1
	
Los average win del estado 0000010-3 son:[0.81, 0.45, 0.33, 0.33, 0.21, 0.3, 0.0, 0.0, 0.0, 0.0]
	
Si apuesto con el estado 0000010-3. Su average win es 0.81 y su auto cashout es 1.1
win or lose es:[1, 1, 0, 0, 0, 0, 0, 0, 0, 0]  total_multiplicador es: -56 el odd fue: 1.24
el estado 0000010-3 tenía un average win de: 0.81 y pasó a ser: 0.85
1.2
	
Los average win del estado 0000001-3 son:[1.0, 0.76, 0.66, 0.2, 0.2, 0.26, 0.04, 0.04, 0.04, 0.04]
	
Si apuesto con el estado 0000

win or lose es:[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]  total_multiplicador es: -35 el odd fue: 3.11
el estado 0011001-3 tenía un average win de: 0.90 y pasó a ser: 0.94
	
Los average win del estado 1001100-3 son:[0.2, 0.12, 0.08, 0.04, 0.04, 0.04, 0.04, 0.04, 0.0, 0.0]
	
No apuesto con el estado 1001100-3
win or lose es:[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  total_multiplicador es: -36 el odd fue: 1.09
el estado 1001100-3 tenía un average win de: 0.20 y pasó a ser: 0.16
1.1
	
Los average win del estado 0100110-3 son:[0.84, 0.64, 0.46, 0.38, 0.28, 0.4, 0.08, 0.08, 0.04, 0.04]
	
Si apuesto con el estado 0100110-3. Su average win es 0.84 y su auto cashout es 1.1
win or lose es:[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  total_multiplicador es: -37 el odd fue: 1.06
el estado 0100110-3 tenía un average win de: 0.84 y pasó a ser: 0.76
1.6
	
Los average win del estado 0010011-3 son:[1.0, 0.95, 0.85, 0.77, 0.67, 0.85, 0.54, 0.34, 0.28, 0.2]
	
Si apuesto con el estado 0010011-3. Su average win es 0.85 y su auto cashout e

...

win or lose es:[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]  total_multiplicador es: -38 el odd fue: 123.58
el estado 0010100-3 tenía un average win de: 0.83 y pasó a ser: 0.87
1.1
	
Los average win del estado 1001010-3 son:[0.84, 0.65, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0]
	
Si apuesto con el estado 1001010-3. Su average win es 0.84 y su auto cashout es 1.1
win or lose es:[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]  total_multiplicador es: -28 el odd fue: 123.58
el estado 0010100-3 tenía un average win de: 0.87 y pasó a ser: 0.91
1.1
	
Los average win del estado 1001010-3 son:[0.84, 0.65, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0]
	
Si apuesto con el estado 1001010-3. Su average win es 0.84 y su auto cashout es 1.1
win or lose es:[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]  total_multiplicador es: -18 el odd fue: 123.58
el estado 0010100-3 tenía un average win de: 0.91 y pasó a ser: 0.95
1.1
	
Los average win del estado 1001010-3 son:[0.84, 0.65, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0]
	
Si apuesto con el estado 1001010-3. S

win or lose es:[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  total_multiplicador es: 37 el odd fue: 1.00
el estado 1110000-3 tenía un average win de: 0.92 y pasó a ser: 0.84
1.2
	
Los average win del estado 0111000-3 son:[0.88, 0.8, 0.7, 0.26, 0.14, 0.15, 0.0, 0.0, 0.0, 0.0]
	
Si apuesto con el estado 0111000-3. Su average win es 0.8 y su auto cashout es 1.2
win or lose es:[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]  total_multiplicador es: 36 el odd fue: 4.36
el estado 0111000-3 tenía un average win de: 0.88 y pasó a ser: 0.92
1.2
	
Los average win del estado 1011100-3 son:[0.99, 0.87, 0.67, 0.41, 0.24, 0.5, 0.24, 0.12, 0.12, 0.12]
	
Si apuesto con el estado 1011100-3. Su average win es 0.87 y su auto cashout es 1.2
win or lose es:[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]  total_multiplicador es: 35 el odd fue: 4.35
el estado 1011100-3 tenía un average win de: 0.99 y pasó a ser: 1.00
	
Los average win del estado 1101110-3 son:[0.55, 0.48, 0.48, 0.16, 0.08, 0.42, 0.08, 0.0, 0.0, 0.0]
	
No apuesto con el estado 1101110-3
wi

win or lose es:[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]  total_multiplicador es: 46 el odd fue: 1.16
el estado 0011001-3 tenía un average win de: 1.00 y pasó a ser: 1.00
	
Los average win del estado 0001100-3 son:[0.4, 0.32, 0.32, 0.24, 0.24, 0.24, 0.24, 0.16, 0.16, 0.16]
	
No apuesto con el estado 0001100-3
win or lose es:[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]  total_multiplicador es: 56 el odd fue: 78.86
el estado 0001100-3 tenía un average win de: 0.40 y pasó a ser: 0.44
1.1
	
Los average win del estado 1000110-3 son:[0.7, 0.64, 0.54, 0.36, 0.28, 0.48, 0.1, 0.08, 0.08, 0.08]
	
Si apuesto con el estado 1000110-3. Su average win es 0.7 y su auto cashout es 1.1
win or lose es:[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]  total_multiplicador es: 55 el odd fue: 6.05
el estado 1000110-3 tenía un average win de: 0.70 y pasó a ser: 0.74
1.2
	
Los average win del estado 1100011-3 son:[1.0, 1.0, 0.39, 0.08, 0.04, 0.19, 0.04, 0.04, 0.04, 0.04]
	
Si apuesto con el estado 1100011-3. Su average win es 1.0 y su auto cashout es 1

	
Los average win del estado 0101111-3 son:[0.77, 0.61, 0.49, 0.16, 0.16, 0.54, 0.08, 0.08, 0.08, 0.08]
	
Si apuesto con el estado 0101111-3. Su average win es 0.77 y su auto cashout es 1.1
win or lose es:[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]  total_multiplicador es: 87 el odd fue: 2.30
el estado 0101111-3 tenía un average win de: 0.77 y pasó a ser: 0.81
1.1
	
Los average win del estado 1010111-3 son:[0.77, 0.53, 0.45, 0.45, 0.37, 0.51, 0.12, 0.12, 0.12, 0.12]
	
Si apuesto con el estado 1010111-3. Su average win es 0.77 y su auto cashout es 1.1
win or lose es:[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]  total_multiplicador es: 97 el odd fue: 13.94
el estado 1010111-3 tenía un average win de: 0.77 y pasó a ser: 0.81
	
Los average win del estado 1101011-3 son:[0.4, 0.32, 0.24, 0.16, 0.16, 0.16, 0.16, 0.08, 0.08, 0.08]
	
No apuesto con el estado 1101011-3
win or lose es:[1, 1, 0, 0, 0, 0, 0, 0, 0, 0]  total_multiplicador es: 96 el odd fue: 1.22
el estado 1101011-3 tenía un average win de: 0.40 y pasó a ser: 

	
Los average win del estado 1010011-3 son:[0.81, 0.67, 0.55, 0.55, 0.55, 0.92, 0.55, 0.31, 0.31, 0.31]
	
Si apuesto con el estado 1010011-3. Su average win es 0.92 y su auto cashout es 1.6
win or lose es:[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  total_multiplicador es: 74 el odd fue: 1.10
el estado 1010011-3 tenía un average win de: 0.81 y pasó a ser: 0.73
	
Los average win del estado 0101001-3 son:[0.57, 0.57, 0.41, 0.27, 0.27, 0.61, 0.08, 0.04, 0.0, 0.0]
	
No apuesto con el estado 0101001-3
win or lose es:[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]  total_multiplicador es: 73 el odd fue: 5.56
el estado 0101001-3 tenía un average win de: 0.57 y pasó a ser: 0.61
1.2
	
Los average win del estado 1010100-3 son:[0.96, 0.84, 0.6, 0.24, 0.24, 0.68, 0.03, 0.03, 0.03, 0.0]
	
Si apuesto con el estado 1010100-3. Su average win es 0.84 y su auto cashout es 1.2
win or lose es:[1, 1, 1, 1, 1, 1, 1, 0, 0, 0]  total_multiplicador es: 72 el odd fue: 1.78
el estado 1010100-3 tenía un average win de: 0.96 y pasó a ser: 1.00

In [ ]:
#def thread_function(elemento_inicial_texto):
#    logging.info("thread to verify if se puteó is starting") #imprimir log
#
#    global driver
#    global puedo_retirar
#    puedo_retirar = True
#    wait = WebDriverWait(driver, 600)
#    element = wait.until(history_has_changed( elemento_inicial_texto))
#    puedo_retirar = True
#    global stop_threads
#    while  (element == None) and (stop_threads==False):
#        logging.info("estoy en el while")
#        time.sleep(0.5)
#        if stop_threads: 
#            print("paró")
#            break
#    puedo_retirar = False
#    
#    logging.info("thread to verify if se puteó is finishing")            
#    
#    #Crear thread
            #if __name__ == "__main__":

             #   format = "%(asctime)s: %(message)s"
              #  logging.basicConfig(format=format, level=logging.INFO,datefmt="%H:%M:%S")

               # global stop_threads
                #stop_threads = False
                #x = threading.Thread(target=thread_function, args=(elemento_inicial_texto,))

                #x.start()
                #logging.info("Main    : wait for the thread to finish")


            #
            
                        #odds_before_fail = driver.find_element_by_class_name(crash_history)
            #odds_before_fail_text=odds_before_fail.text
            
            #time.sleep(15)
            
            #print("puedo retirar " + str(puedo_retirar))
            #if odds_before_fail_text == odds_before_fail.text and puedo_retirar:
             #   place_bet_button.click()
              #  print("retiro")
                
               # stop_threads = True
               # x.join() #Este comando es clave para forzar la terminación del thread
                #print("el thread está vivo despues de retirar? " + str(x.is_alive()) )

                #print(odds_before_fail_text[:6] +" "+ odds_before_fail.text[:6])
            #else:
                #print("Se putió, hago reset")
                #continue